In [1]:
import nltk
import pymorphy2
import re

from tqdm import tqdm
from os import listdir
from os.path import isfile, join

In [2]:
inp_path = ".\\data\\some_data\\raw_data\\"

In [3]:
files = [file for file in listdir(inp_path) if isfile(join(inp_path, file))]

In [4]:
texts = []

In [5]:
for file in tqdm(files):
    with open(inp_path + file, 'r', encoding='utf-8') as f:
        texts.extend(f.readlines())

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 16.04it/s]


In [6]:
texts = [text.translate({ord('\n'): None}) for text in tqdm(texts)]

100%|██████████████████████████████████████████████████████████████████████| 360278/360278 [00:00<00:00, 445393.23it/s]


In [7]:
lemmatized_texts = []

In [8]:
morph = pymorphy2.MorphAnalyzer()

In [9]:
for text in tqdm(texts):
    tokenized_text = nltk.wordpunct_tokenize(text)
    lemmatized_text = [morph.parse(word)[0].normal_form for word in tokenized_text]
    lemmatized_text = ' '.join([str(word) for word in lemmatized_text])
    lemmatized_text = re.sub(r'\s([?.!"](?:\s|$))', r'\1', lemmatized_text)
    lemmatized_texts.append(lemmatized_text)

100%|████████████████████████████████████████████████████████████████████████| 360278/360278 [00:40<00:00, 8830.67it/s]


In [10]:
out_path = ".\\data\\some_data\\processed_data\\"

In [11]:
with open(out_path + "original_texts.txt", 'w', encoding='utf-8') as f:
    f.writelines([text + '\n' for text in tqdm(texts)])

100%|█████████████████████████████████████████████████████████████████████| 360278/360278 [00:00<00:00, 3284048.39it/s]


In [12]:
with open(out_path + "lemmatized_texts.txt", 'w', encoding='utf-8') as f:
    f.writelines([lemmatized_text + '\n' for lemmatized_text in tqdm(lemmatized_texts)])

100%|█████████████████████████████████████████████████████████████████████| 360278/360278 [00:00<00:00, 2767058.33it/s]
